In [26]:
import os
import argparse
import shutil
import numpy as np
import pandas as pd
import json
import copy
import random
import math
import time
import torch
from data_loader import get_test_loader
import config
from models import VqaClassifierModel
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")





    
# if __name__ == '__main__':
#     main()

In [27]:
test_df_res=None

In [28]:
def inference(model, test_loader, answer_classes_dict, path_change):
    global test_df_res
    since = time.time()
    model.eval()
    results = []
    res={'Image_id':[],'Answer': [] }
    print('Inferencing ...')
    # Iterate over data.
    # print('len',len(test_loader))
    # print('loop')
    for batch_idx, batch_sample in enumerate(test_loader):
        # print('batch_sample',batch_sample)
        image = batch_sample['image_feature'].to(device) 
        image_id = batch_sample['image_id']
        questions = batch_sample['question']
        # answers = batch_sample['answer']
        output = model(image, questions)
        preds = torch.argmax(output, dim=-1)
        preds = preds.cpu().detach().numpy()

        # print('preds',type(preds))
        # print('preds',preds)
        # print('image_id',image_id)
        assert (len(preds) == len(image_id))

        ans_keys = list(answer_classes_dict.keys())
        ans_values = list(answer_classes_dict.values())
    

        for pred, image_name in zip(preds, image_id):
            # print('pred',pred)
            # print('image_name',image_name)
            index_ans = ans_values.index(pred)
            results.append({image_name+'|'+ans_keys[index_ans]})
            res['Image_id'].append(image_name)
            res['Answer'].append(ans_keys[index_ans])

    df = pd.DataFrame(results)
    test_df_res = pd.DataFrame(res)
    # print('df')
    # print(df)
    df.to_csv(path_change+'/submission.csv', index=False)

    time_elapsed = time.time() - since
    print('Evaluation complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))


In [29]:
import easydict

In [30]:
with open(config.path+'/VQA_Med/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)


l = len(answer_classes) 

print(l)

457


In [31]:
vqa = np.load('/home/arunava/VQA_Med/test_dataset_pickle/test_dataset_df.pkl', allow_pickle=True )

In [32]:
vqa

image_id                                         image_path  \
0    synpic54082  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
1    synpic48556  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
2    synpic50696  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
3    synpic37194  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
4    synpic31308  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
..           ...                                                ...   
495  synpic21789  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
496  synpic39878  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
497  synpic41525  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
498  synpic18173  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   
499  synpic54143  /home/arunava/VQA_Med_2019_Dataset/Test/VQAMed...   

                                            Question Group  
0                           what modality is shown?   test  
1                  was the mri taken with contrast?   test  
2      what type of contrast did this patient have?   test  
3                     what imaging method was used?   test  
4                           what modality is shown?   test  
..                                               ...   ...  
495                what is abnormal in the ct scan?   test  
496           what is most alarming about this mri?   test  
497       what is most alarming about this ct scan?   test  
498  what is the primary abnormality in this image?   test  
499  what is the primary abnormality in this image?   test  

[500 rows x 4 columns]

In [33]:
def main():

   
    path_output_change =config.path_output_chd
    path_change = config.path

    input_dir = config.path_output_chd
    input_test = config.input_test
    img_feat_test = config.img_feat_test

    with open(path_output_change+'/answer_classes.json', 'r') as j:
        answer_classes = json.load(j)

    # opt = config.parse_opt()

    opt = easydict.EasyDict({
        "SEED":97,
        "BATCH_SIZE": 64,
        "VAL_BATCH_SIZE": 64,
        "NUM_OUTPUT_UNITS": l,
        "MAX_QUESTION_LEN": 17,
        "IMAGE_CHANNEL": 1984,
        "INIT_LERARNING_RATE": 1e-4,
        "LAMNDA":0.0001,
        "MFB_FACTOR_NUM":5,
        "MFB_OUT_DIM":1000,
        "BERT_UNIT_NUM":768,
        "BERT_DROPOUT_RATIO":0.3,
        "MFB_DROPOUT_RATIO":0.1,
        "NUM_IMG_GLIMPSE":2,
        "NUM_QUESTION_GLIMPSE":2,
        "IMG_FEAT_SIZE":1,
        "IMG_INPUT_SIZE":224,
        "NUM_EPOCHS":200,
    })
        
    
    batch_size = opt.BATCH_SIZE
    num_workers = 0
    image_size = opt.IMG_INPUT_SIZE
    
    # print('imput_test',input_test)
    # Create the DataLoader for our dataset
    test_data_loader = get_test_loader(
        input_dir = input_dir , 
        input_test = input_test, 
        img_feat_vqa = img_feat_test,
        batch_size = batch_size, 
        num_workers = num_workers,
        size = image_size )

    # print('test_data_loader',test_data_loader)

    model = VqaClassifierModel( opt=opt ).to(device)
    saved_dir = config.path_output_chd
    filename =saved_dir+'/mixed_model_state_seed_97.tar'
    print("=> loading checkpoint '{}'".format(filename)) 
    checkpoint = torch.load(filename) 
    start_epoch = checkpoint['epoch'] 
    model.load_state_dict(checkpoint['state_dict']) 
  
    print("=> loaded checkpoint '{}' (epoch {})" .format(filename, checkpoint['epoch']))
    inference(model=model, test_loader=test_data_loader, answer_classes_dict=answer_classes , path_change= path_change)

In [34]:
main()

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


=> loading checkpoint '/home/arunava/VQA_Med/model_state_seed_97.tar'
=> loaded checkpoint '/home/arunava/VQA_Med/model_state_seed_97.tar' (epoch 139)
Inferencing ...


/home/arunava/miniconda3/envs/vqamedenv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Evaluation complete in 0m 0s


In [35]:
test_df_res

Image_id              Answer
0    synpic61010  acute appendicitis
1    synpic29662  acute appendicitis
2    synpic28805             UNKNOWN
3    synpic27965             UNKNOWN
4    synpic27041             UNKNOWN
..           ...                 ...
495  synpic16279             UNKNOWN
496  synpic15867             UNKNOWN
497  synpic21107             UNKNOWN
498  synpic29722             UNKNOWN
499  synpic45113  acute appendicitis

[500 rows x 2 columns]

In [36]:
test_df_res.shape

(500, 2)

In [37]:
test_ref_path_2020 = config.path + '/VQA_Med_Dataset' + '/test_2020/Task1-2020-VQAnswering-Test-Questions-Answers.csv'
test_ref_path_2019 = config.path + '/VQA_Med_Dataset' + '/test_2019/VQAMed2019_Test_Questions_w_Ref_Answers.txt'

dict_data = {'Image_id':[],
            'Category':[],
            'Question':[],
            'Answer':[]
            }


with open(test_ref_path_2019) as f:
    lines = f.readlines()

    for element in lines:
        # print('element',element)
        pd_element = element.split('|')
        # print('pd_element',pd_element[0])
        dict_data['Image_id'].append(pd_element[0])
        dict_data['Category'].append(pd_element[1])
        dict_data['Question'].append(pd_element[2])
        dict_data['Answer'].append(pd_element[3].strip("\n"))




In [38]:
print(len(dict_data['Image_id']))
print(len(dict_data['Question']))
print(len(dict_data['Answer']))


500
500
500


In [39]:
with open(test_ref_path_2020) as f:
    lines = f.readlines()

    for element in lines:
        # print('element',element)
        pd_element = element.split(',')
        # print('pd_element',pd_element[0])
        dict_data['Image_id'].append(pd_element[0])
        # dict_data['Category'].append(pd_element[1])
        dict_data['Question'].append(pd_element[1])
        dict_data['Answer'].append(pd_element[2].strip("\n"))



In [40]:
print(len(dict_data['Image_id']))
print(len(dict_data['Question']))
print(len(dict_data['Answer']))

843
843
843


In [41]:
test_df = pd.DataFrame(dict_data,columns=['Image_id','Question','Answer'])

test_df

Image_id                                        Question  \
0    synpic54082                         what modality is shown?   
1    synpic48556                was the mri taken with contrast?   
2    synpic50696    what type of contrast did this patient have?   
3    synpic37194                   what imaging method was used?   
4    synpic31308                         what modality is shown?   
..           ...                                             ...   
838  synpic23059          what abnormality is seen in the image?   
839  synpic15170         what is most alarming about this x-ray?   
840  synpic33916  what is the primary abnormality in this image?   
841  synpic51296       what is most alarming about this ct scan?   
842  synpic27946  what is the primary abnormality in this image?   

                        Answer  
0         cta - ct angiography  
1                           no  
2                           iv  
3    us-d - doppler ultrasound  
4               an - angiogram  
..                         ...  
838             angiomyolipoma  
839         pulmonary embolism  
840                adenomyosis  
841             cholelithiasis  
842         pulmonary embolism  

[843 rows x 3 columns]

In [42]:
import pickle


# d = pickle.load('./test_dataset_pickle/C1_test_dataset_df.pkl')
path_change = config.path
test_path = path_change + '/VQA_Med_2019_Dataset/Test/VQAMed2019Test'
# test_ref_path = test_path + '/VQAMed2019_Test_Questions_w_Ref_Answers.txt'

test_ref_path = '/home/arunava/VQA_Med_Dataset/val_2021/All_QA_Pairs_test.txt'
with open(test_ref_path) as f:
    lines = f.readlines()

dict_data = {'Image_id':[],
            # 'Category':[],
            'Question':[],
            'Answer':[]
            }

for element in lines:
    # print('element',element)
    pd_element = element.split('|')
    # print('pd_element',pd_element[0])
    dict_data['Image_id'].append(pd_element[0])
    # dict_data['Category'].append(pd_element[1])
    dict_data['Question'].append(pd_element[1])
    dict_data['Answer'].append(pd_element[2].strip("\n"))


test_df = pd.DataFrame(dict_data,columns=['Image_id','Category','Question','Answer'])

test_df



In [43]:
test_df.shape

(843, 3)

In [44]:
pred = test_df_res['Answer'].to_list()
pred

['acute appendicitis',
 'acute appendicitis',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'acute appendicitis',
 'UNKNOWN',
 'UNKNOWN',
 'sarcoidosis',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'synovial cyst',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN',
 'UNKNOWN

In [45]:
truelabels = test_df['Answer'].to_list()
truelabels

['cta - ct angiography',
 'no',
 'iv',
 'us-d - doppler ultrasound',
 'an - angiogram',
 't2',
 'yes',
 'gi and iv',
 'yes',
 'yes',
 'mr - flair',
 'no',
 'yes',
 'xr - plain film',
 'yes',
 'flair',
 'xr - plain film',
 'ct with iv contrast',
 'mr - t1w w/gadolinium',
 'yes',
 'no',
 'yes',
 'yes',
 't2',
 'no',
 'no',
 'no',
 'xr - plain film',
 'mr - flair',
 'ct with gi and iv contrast',
 'xr - plain film',
 'contrast',
 'yes',
 'us - ultrasound',
 't2',
 'ct w/contrast (iv)',
 'no',
 'ct w/contrast (iv)',
 'no',
 'no',
 'yes',
 'no',
 'an - angiogram',
 'us - ultrasound',
 'no',
 'no',
 'no',
 'cta - ct angiography',
 'iv',
 'yes',
 'yes',
 'noncontrast',
 'mr - flair',
 'contrast',
 'an - angiogram',
 'xr - plain film',
 'yes',
 'mammograph',
 'no',
 'yes',
 'an - angiogram',
 'ugi - upper gi',
 'yes',
 'mr - t2 weighted',
 'noncontrast (mri)',
 'yes',
 'contrast',
 'ct with gi and iv contrast',
 'us - ultrasound',
 'no',
 'ct noncontrast',
 'contrast',
 'xr - plain film',
 'mr 

In [46]:
cnt=0
for i in range(len(pred)):
    if(pred[i] == truelabels[i]): cnt=cnt+1

cnt

0

In [47]:
len(pred)

500

In [48]:
(cnt/len(pred))*100

0.0

In [49]:
image_ids = test_df_res['Image_id'].to_list()
preds = test_df_res['Answer'].to_list()
questions = dict_data['Question']
answers = dict_data['Answer']
Image_ids = dict_data['Image_id']
categories = dict_data['Category']

for i in range(len(image_ids)):
    if image_ids[i] != Image_ids[i]:
        print('mismatch at index',i)
        break


cnt=0
correct ={'modality': 0, 'plane': 0, 'organ': 0, 'abnormality': 0}
count ={'modality': 0, 'plane': 0, 'organ': 0, 'abnormality': 0}

for i in range(len(image_ids)):
    if (preds[i] == answers[i]): 
        cnt = cnt+1
        correct[categories[i]] = correct[categories[i]]+1
    count[categories[i]] = count[categories[i]]+1

mismatch at index 0


In [50]:
print('cnt',cnt)
print('correct',correct)
print('count',count)

cnt 0
correct {'modality': 0, 'plane': 0, 'organ': 0, 'abnormality': 0}
count {'modality': 125, 'plane': 125, 'organ': 125, 'abnormality': 125}
